<a href="https://colab.research.google.com/github/Aditya-Walia1/Bert-Research/blob/main/Optimised_Roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Increased Epochs: More training iterations (num_train_epochs=4).
Reduced Batch Size: Smaller batches for better generalization (per_device_train_batch_size=8).
Lower Learning Rate: Finer weight adjustments (learning_rate=2e-5).
Increased Weight Decay: To prevent overfitting (weight_decay=0.02).
Evaluation and Save Strategy: Both set to 'epoch' for consistency.
Limited Saved Checkpoints: Keeps only the best 2 models.

In [2]:
# Install necessary libraries
!pip install transformers==4.31.0 scikit-learn

from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from google.colab import files
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pickle

# Upload the dataset
uploaded = files.upload()
file_name = next(iter(uploaded))

# Load your dataset
df = pd.read_csv(file_name)

# Inspect the column names to identify the text and label columns
print(df.columns)

# Update these variables with the actual column names in your dataset
text_column = 'tweet'  # Replace with the actual text column name
label_column = 'label'  # Replace with the actual label column name

# Encode the labels as integers
label_encoder = LabelEncoder()
df[label_column] = label_encoder.fit_transform(df[label_column])

# Verify the encoding (optional)
print("Encoded labels:", label_encoder.classes_)

# Preprocess the dataset
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Load RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Split the dataset into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df[text_column].values,
    df[label_column].values,
    test_size=0.2,
    random_state=42
)

# Define max token length
MAX_LEN = 128

# Create train and validation datasets
train_dataset = CustomDataset(train_texts, train_labels, tokenizer, MAX_LEN)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer, MAX_LEN)

# Load pre-trained RoBERTa model for sequence classification
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(df[label_column].unique()))

# Set up optimized training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,  # Increased epochs
    per_device_train_batch_size=8,  # Reduced batch size
    per_device_eval_batch_size=8,
    warmup_steps=200,  # Reduced warmup steps
    weight_decay=0.02,  # Increased weight decay
    learning_rate=2e-5,  # Reduced learning rate for finer adjustments
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",  # Make save strategy match the evaluation strategy
    save_total_limit=2,  # Save only the best 2 models
    load_best_model_at_end=True  # Load the best model based on evaluation
)


# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()

# Prediction for validation data
predictions = trainer.predict(val_dataset)
preds = torch.argmax(torch.tensor(predictions.predictions), axis=1)

# Calculate and print detailed classification metrics
accuracy = accuracy_score(val_labels, preds)
conf_matrix = confusion_matrix(val_labels, preds)
class_report = classification_report(val_labels, preds, target_names=label_encoder.classes_)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

# Optional: Save the label encoder for later use
with open('label_encoder.pkl', 'wb') as le_file:
    pickle.dump(label_encoder, le_file)


Saving Constraint_Train.csv to Constraint_Train (2).csv
Index(['id', 'tweet', 'label'], dtype='object')
Encoded labels: ['fake' 'real']


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.197200,0.141182
2,0.003200,0.141476
3,0.000200,0.177822
4,0.000100,0.173811


Accuracy: 0.9688473520249221
Confusion Matrix:
 [[575  21]
 [ 19 669]]
Classification Report:
               precision    recall  f1-score   support

        fake       0.97      0.96      0.97       596
        real       0.97      0.97      0.97       688

    accuracy                           0.97      1284
   macro avg       0.97      0.97      0.97      1284
weighted avg       0.97      0.97      0.97      1284

